In [1]:
# 📁 Upload train.csv before running this
from google.colab import files
uploaded = files.upload()

Saving test.csv to test.csv
Saving train.csv to train.csv


In [2]:
! pip install -q catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 9.9 MB/s eta 0:00:00


In [3]:
import pandas as pd
import numpy as np
from catboost import CatBoostRegressor
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_percentage_error, r2_score

# === Load Data ===
df_train = pd.read_csv("train.csv")
df_test = pd.read_csv("test.csv")

features = df_train.columns[:55].tolist()
targets = df_train.columns[55:].tolist()

X_train_raw = df_train[features]
X_test_raw = df_test[features].iloc[:500]
Y_train = df_train[targets]

# === Impute + Scale ===
imp = SimpleImputer()
X_train = imp.fit_transform(X_train_raw)
X_test = imp.transform(X_test_raw)

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

df_train_scaled = pd.DataFrame(X_train, columns=features)
df_test_scaled = pd.DataFrame(X_test, columns=features)

# === Tuned Parameters per BlendProperty ===
param_grid = {
    'BlendProperty1': {'depth': 6, 'learning_rate': 0.05, 'iterations': 2700, 'l2_leaf_reg': 4},
    'BlendProperty2': {'depth': 5, 'learning_rate': 0.06, 'iterations': 2600, 'l2_leaf_reg': 3},
    'BlendProperty3': {'depth': 6, 'learning_rate': 0.045, 'iterations': 3000, 'l2_leaf_reg': 5},
    'BlendProperty4': {'depth': 4, 'learning_rate': 0.05, 'iterations': 2500, 'l2_leaf_reg': 4},
    'BlendProperty5': {'depth': 7, 'learning_rate': 0.035, 'iterations': 2800, 'l2_leaf_reg': 6},
    'BlendProperty6': {'depth': 6, 'learning_rate': 0.04, 'iterations': 2900, 'l2_leaf_reg': 5},
    'BlendProperty7': {'depth': 5, 'learning_rate': 0.06, 'iterations': 2500, 'l2_leaf_reg': 3},
    'BlendProperty8': {'depth': 6, 'learning_rate': 0.045, 'iterations': 2700, 'l2_leaf_reg': 4},
    'BlendProperty9': {'depth': 5, 'learning_rate': 0.05, 'iterations': 2600, 'l2_leaf_reg': 4},
    'BlendProperty10': {'depth': 6, 'learning_rate': 0.04, 'iterations': 3000, 'l2_leaf_reg': 5},
}

final_preds = []
metrics_log = []

for col in targets:
    print(f"\n🔁 Processing {col}")
    y_col = Y_train[col].values
    p = param_grid[col]

    # === Feature Selection using a base model ===
    base_model = CatBoostRegressor(
        iterations=p['iterations'],
        learning_rate=p['learning_rate'],
        depth=p['depth'],
        l2_leaf_reg=p['l2_leaf_reg'],
        bagging_temperature=0.7,
        random_seed=42,
        loss_function='RMSE',
        od_type="Iter",
        od_wait=50,
        verbose=0
    )
    base_model.fit(X_train, y_col)
    importance_df = base_model.get_feature_importance(prettified=True)
    important_feature_ids = importance_df[importance_df['Importances'] > 0.1]['Feature Id'].astype(int).tolist()
    important_feature_names = [features[i] for i in important_feature_ids]

    if not important_feature_names:
        print(f"⚠ No important features for {col}. Using all features.")
        important_feature_names = features

    X_train_sub = df_train_scaled[important_feature_names].values
    X_test_sub = df_test_scaled[important_feature_names].values

    preds_test_all = []
    preds_train_all = []

    for seed_offset in [0, 11, 27]:  # 3 different seeds/models
        model = CatBoostRegressor(
            iterations=p['iterations'],
            learning_rate=p['learning_rate'],
            depth=p['depth'],
            l2_leaf_reg=p['l2_leaf_reg'],
            bagging_temperature=0.7,
            random_seed=42 + seed_offset,
            loss_function='RMSE',
            od_type="Iter",
            od_wait=50,
            verbose=0
        )
        model.fit(X_train_sub, y_col)
        preds_test_all.append(model.predict(X_test_sub))
        preds_train_all.append(model.predict(X_train_sub))

    # === Average Predictions ===
    y_pred_test = np.mean(preds_test_all, axis=0)
    y_pred_train = np.mean(preds_train_all, axis=0)

    final_preds.append(y_pred_test)

    r2 = r2_score(y_col, y_pred_train)
    mape = mean_absolute_percentage_error(y_col, y_pred_train)
    metrics_log.append((col, r2, mape))
    print(f"{col} ✅ R²: {r2:.4f}, MAPE: {mape:.4f}")

# === Save Final Predictions ===
pred_df = pd.DataFrame(np.array(final_preds).T, columns=targets)
pred_df.insert(0, "ID", range(1, 501))
pred_df.to_csv("catboost_ensemble_predictions.csv", index=False)
print("\n📁 Saved: catboost_ensemble_predictions.csv")

# === Summary ===
print("\n📊 Final Scores:")
for name, r2, mape in metrics_log:
    print(f"{name} - R²: {r2:.4f}, MAPE: {mape:.4f}")



🔁 Processing BlendProperty1
BlendProperty1 ✅ R²: 1.0000, MAPE: 0.0299

🔁 Processing BlendProperty2
BlendProperty2 ✅ R²: 1.0000, MAPE: 0.0387

🔁 Processing BlendProperty3
BlendProperty3 ✅ R²: 1.0000, MAPE: 0.0342

🔁 Processing BlendProperty4
BlendProperty4 ✅ R²: 0.9997, MAPE: 0.1036

🔁 Processing BlendProperty5
BlendProperty5 ✅ R²: 1.0000, MAPE: 0.0124

🔁 Processing BlendProperty6
BlendProperty6 ✅ R²: 0.9999, MAPE: 0.0453

🔁 Processing BlendProperty7
BlendProperty7 ✅ R²: 0.9999, MAPE: 0.1396

🔁 Processing BlendProperty8
BlendProperty8 ✅ R²: 1.0000, MAPE: 0.0290

🔁 Processing BlendProperty9
BlendProperty9 ✅ R²: 0.9997, MAPE: 0.0985

🔁 Processing BlendProperty10
BlendProperty10 ✅ R²: 0.9999, MAPE: 0.0537

📁 Saved: catboost_ensemble_predictions.csv

📊 Final Scores:
BlendProperty1 - R²: 1.0000, MAPE: 0.0299
BlendProperty2 - R²: 1.0000, MAPE: 0.0387
BlendProperty3 - R²: 1.0000, MAPE: 0.0342
BlendProperty4 - R²: 0.9997, MAPE: 0.1036
BlendProperty5 - R²: 1.0000, MAPE: 0.0124
BlendProperty6 - 